In [1]:
import cv2
import numpy as np
import sys
import os
import plotly.graph_objects as go
from scipy.spatial.transform import Rotation as R_scipy

def detect_aruco_in_image(
    image_path,
    output_path=None,
    aruco_dict_type=cv2.aruco.DICT_6X6_250,
    marker_size_mm=60.0,
    camera_matrix=None,
    dist_coeffs=None
):
    if camera_matrix is None:
        raise ValueError("必须提供相机内参")
    if dist_coeffs is None:
        dist_coeffs = np.zeros(5, dtype=np.float32)

    image = cv2.imread(image_path)
    if image is None:
        raise FileNotFoundError(f"无法读取图像: {image_path}")
    h, w = image.shape[:2]
    print(f"图像尺寸: {w} × {h}")

    aruco_dict = cv2.aruco.getPredefinedDictionary(aruco_dict_type)
    aruco_params = cv2.aruco.DetectorParameters()
    detector = cv2.aruco.ArucoDetector(aruco_dict, aruco_params)
    corners, ids, rejected = detector.detectMarkers(image)

    output_image = image.copy()
    results = []

    if ids is not None:
        print(f"✅ 检测到 {len(ids)} 个 ArUco 标记: IDs = {ids.flatten().tolist()}")
        marker_size_m = marker_size_mm / 1000.0
        obj_points = np.array([
            [-marker_size_m / 2,  marker_size_m / 2, 0],
            [ marker_size_m / 2,  marker_size_m / 2, 0],
            [ marker_size_m / 2, -marker_size_m / 2, 0],
            [-marker_size_m / 2, -marker_size_m / 2, 0]
        ], dtype=np.float32)

        for i in range(len(ids)):
            marker_id = int(ids[i][0])
            corner = corners[i].reshape((4, 2))
            success, rvec, tvec = cv2.solvePnP(
                obj_points, corner, camera_matrix, dist_coeffs,
                flags=cv2.SOLVEPNP_IPPE_SQUARE
            )
            if not success:
                print(f"⚠️ ID {marker_id} 位姿求解失败")
                continue

            cv2.polylines(output_image, [corner.astype(int)], True, (0, 255, 0), 2)
            cv2.putText(output_image, f"ID:{marker_id}", 
                        tuple(corner[0].astype(int)), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
            cv2.drawFrameAxes(output_image, camera_matrix, dist_coeffs, rvec, tvec, 0.05)
            z_cm = tvec[2][0] * 100
            cv2.putText(output_image, f"Z:{z_cm:.1f}cm",
                        (corner[0][0].astype(int), corner[0][1].astype(int) - 20),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

            R_mat, _ = cv2.Rodrigues(rvec)
            results.append({
                "id": marker_id,
                "tvec": tvec.flatten(),
                "rvec": rvec.flatten(),
                "R": R_mat
            })
            x, y, z = tvec.flatten() * 100
            print(f"\n[ID {marker_id}] 位置 (cm): x={x:.1f}, y={y:.1f}, z={z:.1f}")
    else:
        print("❌ 未检测到任何 ArUco 标记")

    if output_path is None:
        base, ext = os.path.splitext(image_path)
        output_path = f"{base}_detected{ext}"
    cv2.imwrite(output_path, output_image)
    print(f"\n📁 结果图像已保存: {output_path}")
    return results, output_image, corners, ids

def extract_3d_points_from_aruco_masks(
    depth_image,
    corners,
    ids,
    camera_matrix,
    depth_scale=1.0,
    fill_holes=True
):
    if ids is None:
        return {}
    fx, fy = camera_matrix[0, 0], camera_matrix[1, 1]
    cx, cy = camera_matrix[0, 2], camera_matrix[1, 2]
    depth = depth_image.astype(np.float32) * depth_scale
    h, w = depth.shape
    point_clouds = {}
    for i in range(len(ids)):
        marker_id = int(ids[i][0])
        pts = corners[i].reshape((4, 2)).astype(np.int32)
        mask = np.zeros((h, w), dtype=np.uint8)
        if fill_holes:
            cv2.fillPoly(mask, [pts], 255)
        else:
            cv2.polylines(mask, [pts], True, 255, thickness=1)
        valid = (mask > 0) & (depth > 0)
        v_coords, u_coords = np.where(valid)
        if len(u_coords) == 0:
            point_clouds[marker_id] = np.empty((0, 3))
            continue
        Z = depth[v_coords, u_coords]
        X = (u_coords - cx) * Z / fx
        Y = (v_coords - cy) * Z / fy
        points_3d = np.stack((X, Y, Z), axis=-1)
        point_clouds[marker_id] = points_3d
    return point_clouds

def compute_aruco_centers_from_pointclouds(point_clouds):
    centers = {}
    for marker_id, points in point_clouds.items():
        if points.size == 0:
            centers[marker_id] = np.full(3, np.nan)
        else:
            centers[marker_id] = np.mean(points, axis=0)
    return centers

# ==============================
# 用户配置区
# ==============================
IMAGE_PATH = "bagimg/new_img.png"
DEPTH_PATH = "bagimg/new_depth_raw.tiff"
CAMERA_MATRIX = np.array([
    [612.593017578125, 0.0,    639.3008422851562],
    [0.0,    612.551513671875, 405.6265869140625],
    [0.0,    0.0,    1.0]
], dtype=np.float32)
DIST_COEFFS = np.array([0.0, 0.0, 0.0, 0.0, 0.0], dtype=np.float32)

# ==============================
# 主流程
# ==============================
if not os.path.exists(IMAGE_PATH):
    print(f"❌ 图像文件不存在: {IMAGE_PATH}")
    sys.exit(1)

results, out_img, corners, ids = detect_aruco_in_image(
    image_path=IMAGE_PATH,
    aruco_dict_type=cv2.aruco.DICT_6X6_250,
    marker_size_mm=48.3,
    camera_matrix=CAMERA_MATRIX,
    dist_coeffs=DIST_COEFFS
)

# 加载深度图并提取点云（仅用于可视化/对比）
depth = cv2.imread(DEPTH_PATH, cv2.IMREAD_UNCHANGED)
point_clouds = extract_3d_points_from_aruco_masks(
    depth_image=depth,
    corners=corners,
    ids=ids,
    camera_matrix=CAMERA_MATRIX,
    depth_scale=0.001
)

centers = compute_aruco_centers_from_pointclouds(point_clouds)
for marker_id, center in centers.items():
    if not np.any(np.isnan(center)):
        x, y, z = center * 100
        print(f"[ID {marker_id}] 点云几何中心 (cm): x={x:.1f}, y={y:.1f}, z={z:.1f}")

# ==============================
# 基于 ArUco 位姿计算目标位姿（关键部分）
# ==============================
required_ids = [0, 1]
id_to_pose = {}
for res in results:
    if res["id"] in required_ids:
        id_to_pose[res["id"]] = {
            "tvec": res["tvec"],      # (3,) 单位：米
            "R": res["R"]             # (3,3)
        }

if len(id_to_pose) != 2:
    print(f"❌ 必须同时检测到 ID 0 和 ID 1，当前只检测到: {list(id_to_pose.keys())}")
    sys.exit(1)

# 向标记“深处”偏移 9 cm（沿其自身 Z 轴正方向）
offset_m = -0.09  # 注意：ArUco 的 Z 轴指向相机，所以“深处”是 -Z
target_points = {}
for marker_id in required_ids:
    tvec = id_to_pose[marker_id]["tvec"]
    R = id_to_pose[marker_id]["R"]
    z_axis = R @ np.array([0, 0, 1])  # 相机坐标系中的 Z 轴方向
    new_point = tvec + offset_m * z_axis
    target_points[marker_id] = new_point
    print(f"\n[ID {marker_id}] 深处9cm点 (cm): {(new_point * 100).round(1)}")

# 计算中点位置（基于 ArUco 位姿）
p0 = target_points[0]
p1 = target_points[1]
midpoint_cam = (p0 + p1) / 2.0
print(f"\n[中点（相机坐标系下，cm）]: {(midpoint_cam * 100).round(1)}")

# 计算平均姿态（使用四元数 Slerp）
quat0 = R_scipy.from_matrix(id_to_pose[0]["R"]).as_quat()  # [x, y, z, w]
quat1 = R_scipy.from_matrix(id_to_pose[1]["R"]).as_quat()
# 简单平均（适用于小角度差异）或使用 slerp(0.5)
quat_avg = R_scipy.from_quat((quat0 + quat1) / 2).as_quat()
quat_avg /= np.linalg.norm(quat_avg)  # 归一化
R_avg = R_scipy.from_quat(quat_avg).as_matrix()

# 将R avg 绕y轴和z轴旋转180度
r_correction = R_scipy.from_euler('yz', [180, 180], degrees=True).as_matrix()
R_avg = R_avg @ r_correction

print(f"\n[平均旋转矩阵]:\n{R_avg}")

# ==============================
# 转换到机械臂基坐标系
# ==============================
# 手眼标定矩阵：T_camera_in_arm_base （示例，请替换为实际值）
T_cam_in_arm = np.array([
    [0, 0, 1, 0.14],
    [-1, 0, 0, 0],
    [0, -1, 0, -0.25],
    [0, 0, 0, 1]
], dtype=np.float32)

# 构造中点齐次坐标
midpoint_hom = np.hstack([midpoint_cam, 1.0])  # (4,)
midpoint_arm = T_cam_in_arm @ midpoint_hom     # (4,)

# 构造完整位姿（旋转 + 平移）
R_arm = T_cam_in_arm[:3, :3] @ R_avg
t_arm = midpoint_arm[:3]

print(f"\n✅ 机械臂末端目标位姿（基坐标系）:")
print(f"  位置 (cm): {(t_arm * 100).round(1)}")
print(f"  旋转矩阵:\n{R_arm}")

# 可选：转换为四元数（供 MoveIt 使用）
quat_arm = R_scipy.from_matrix(R_arm).as_quat()
print(f"  四元数 (x,y,z,w): {quat_arm.round(4)}")

图像尺寸: 1280 × 800
✅ 检测到 2 个 ArUco 标记: IDs = [0, 1]

[ID 0] 位置 (cm): x=-6.0, y=-5.1, z=28.6

[ID 1] 位置 (cm): x=15.5, y=-4.4, z=28.0

📁 结果图像已保存: bagimg/new_img_detected.png
[ID 0] 点云几何中心 (cm): x=-5.8, y=-5.0, z=28.0
[ID 1] 点云几何中心 (cm): x=15.2, y=-4.3, z=27.4

[ID 0] 深处9cm点 (cm): [-5.4 -5.4 37.6]

[ID 1] 深处9cm点 (cm): [15.4 -4.7 37. ]

[中点（相机坐标系下，cm）]: [ 5.  -5.1 37.3]

[平均旋转矩阵]:
[[ 0.99860003 -0.04639699  0.02540285]
 [ 0.04730824  0.99821214 -0.03653029]
 [-0.02366254  0.03768092  0.99900963]]

✅ 机械臂末端目标位姿（基坐标系）:
  位置 (cm): [ 51.3  -5.  -19.9]
  旋转矩阵:
[[-0.02366254  0.03768092  0.99900963]
 [-0.99860003  0.04639699 -0.02540285]
 [-0.04730824 -0.99821214  0.03653029]]
  四元数 (x,y,z,w): [-0.4726  0.5083 -0.5034  0.5146]


In [2]:
# quat_arm 绕x轴旋转-60度
r_x_neg60 = R_scipy.from_euler('y', 60, degrees=True).as_matrix()
R_arm_corrected = r_x_neg60 @ R_arm
quat_arm_corrected = R_scipy.from_matrix(R_arm_corrected).as_quat()
print(f"  纠正后四元数 (x,y,z,w): {quat_arm_corrected.round(4)}")

  纠正后四元数 (x,y,z,w): [-0.661   0.6975 -0.1997  0.1915]


In [3]:
# t arm z+44
t_arm[2] += 0.40
print(f"  袋子高度纠正后位置 (m): {t_arm}")

  袋子高度纠正后位置 (m): [ 0.5129231  -0.04993375  0.20068055]


In [4]:
# t arm 沿四元数的z轴负方向移动12cm
r_z_neg = R_scipy.from_quat(quat_arm_corrected).as_matrix()
z_axis_arm = r_z_neg @ np.array([0, 0, 1])
t_arm -= 0.12 * z_axis_arm
print(f"  夹爪长度纠正最终位置 (m): {t_arm}")

  夹爪长度纠正最终位置 (m): [ 0.44918618 -0.04688541  0.30230885]
